# Introduction

This notebook demonstrates how to perform phase and electrochemical assessments starting from a VASP calculation using Python Materials Genomics (pymatgen) and the Materials Project database (via the Materials API). These notebooks are described in detail in

    Deng, Z.; Zhu, Z.; Chu, I.-H.; Ong, S. P. Data-Driven First-Principles Methods for the Study and Design of 
    Alkali Superionic Conductors. Chem. Mater. 2017, 29 (1), 281–288 DOI: 10.1021/acs.chemmater.6b02648.
    
If you find these notebooks useful and use the functionality demonstrated, please consider citing the above work.

Let's start by importing some modules and classes that we will be using.

In [3]:
%matplotlib inline

from pymatgen.analysis.reaction_calculator import Reaction
from pymatgen.ext.matproj import MPRester, Composition, Element
from pymatgen.io.vasp import Vasprun
from pymatgen.analysis.phase_diagram import PhaseDiagram, CompoundPhaseDiagram, PDEntry, GrandPotentialPhaseDiagram
#from pymatgen.phasediagram.analyzer import PDAnalyzer
from pymatgen.analysis.phase_diagram import PDPlotter
from pymatgen.entries.computed_entries import ComputedEntry
from pymatgen.entries.compatibility import MaterialsProjectCompatibility
import pymatgen.util.plotting as plty
#import palettable.colorbrewer.diverging
import json
import re
import palettable
import matplotlib as mpl
from pymatgen.ext.cod import COD
cod = COD()





# Preparation

We will first read the results from the *vasprun.xml* output file from our VASP calculations. Only the lowest energy result is used here.

In [4]:
vasprun = Vasprun("./vasprun.xml")
# include structure so proper correction can be applied for oxides and sulfides
entry = vasprun.get_computed_entry(inc_structure=True)  


FileNotFoundError: [Errno 2] No such file or directory: './vasprun.xml'

To construct the phase diagram, we need all entries in the compounds chemical space. We will use the *MPRester* class to obtain these entries from the Materials Project via the Materials API.

In [ ]:
rester = MPRester("api_key")
mp_entries = rester.get_entries_in_chemsys(["Li", "In", "Cl"])

In addition to all the MP entries, here we also load the computed entries of Li-Al-O tenary compounds.

Next, we need to combine all the entries and postprocess them using *MaterialsProjectCompatibility*. This postprocessing step corrects the energies to account for well-known DFT errors, e.g., in the sulfur binding energy.

In [ ]:
compatibility = MaterialsProjectCompatibility()
entry = compatibility.process_entry(entry)
entries = compatibility.process_entries([entry] + mp_entries )

# Phase diagram construction

The phase diagram can then be constructed using the *PhaseDiagram* class, and plotted using the *PDPlotter* class.

In [ ]:
pd = PhaseDiagram(entries)
plotter = PDPlotter(pd)
plotter.show()

##### Calculating $E_{\rm hull}$ of Structure

We may evaluate the $E_{\rm hull}$ of the structure using the *PDAnalyzer*.

In [ ]:
ehull = pd.get_e_above_hull(entry)
print("The energy above hull is %.3f eV/atom." % ehull)

# Electrochemical Stability

The electrochemical stability can be assessed using a similar phase diagram approach, but using the lithium grand potential instead of the internal energy.

First, we need to identify a reference for lithium chemical potential using the bulk Li energy $\mu_{\rm Li}^0$.

In [ ]:
Na_entries = [e for e in entries if e.composition.reduced_formula == "Li"]
uLi0 = min(Na_entries, key=lambda e: e.energy_per_atom).energy_per_atom

##### The *PDAnalyzer* class provides a quick way to plot the phase diagram at a particular composition (e.g., LiAl5O8) as a function of lithium chemical potential called *get_element_profile*.

In [ ]:
el_profile = pd.get_element_profile(Element("Li"), entry.composition)
for i, d in enumerate(el_profile):
    voltage = -(d["chempot"] - uLi0)
    print("Voltage: %s V" % voltage)
    print(d["reaction"])
    print("")

This element profile can be plotted as a Li evolution versus voltage using matplotlib as follows.

In [ ]:
# Some matplotlib settings to improve the look of the plot.
mpl.rcParams['axes.linewidth']=3
mpl.rcParams['lines.markeredgewidth']=4
mpl.rcParams['lines.linewidth']=3
mpl.rcParams['lines.markersize']=15
mpl.rcParams['xtick.major.width']=3
mpl.rcParams['xtick.major.size']=8
mpl.rcParams['xtick.minor.width']=3
mpl.rcParams['xtick.minor.size']=4
mpl.rcParams['ytick.major.width']=3
mpl.rcParams['ytick.major.size']=8
mpl.rcParams['ytick.minor.width']=3
mpl.rcParams['ytick.minor.size']=4


# Plot of Na uptake per formula unit (f.u.) of Na2ZrCl6 against voltage vs Na/Na+.

colors = palettable.colorbrewer.qualitative.Set1_9.mpl_colors
plt = plty.pretty_plot(12, 8)

for i, d in enumerate(el_profile):
    v = - (d["chempot"] - uLi0)
    if i != 0:
        plt.plot([x2, x2], [y1, d["evolution"] / 4.0], 'k', linewidth=3)
    x1 = v
    y1 = d["evolution"] / 4.0
    if i != len(el_profile) - 1:
        x2 = - (el_profile[i + 1]["chempot"] - uLi0)
    else:
        x2 = 5.0
        
    if i in [0, 4, 5, 7]:
        products = [re.sub(r"(\d+)", r"$_{\1}$", p.reduced_formula)                     
                    for p in d["reaction"].products if p.reduced_formula != "Li"]

        plt.annotate(", ".join(products), xy=(v + 0.05, y1 + 0.3), 
                     fontsize=24, color=colors[0])
        
        plt.plot([x1, x2], [y1, y1], color=colors[0], linewidth=5)
    else:
        plt.plot([x1, x2], [y1, y1], 'k', linewidth=3)  

plt.xlim((0, 6.0))
plt.ylim((-4, 30))
plt.xlabel("Voltage vs Li/Li$^+$ for Li3InCl6  (V)")
plt.ylabel("Li uptake per f.u.")
plt.tight_layout()